<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/AnalyzeSlate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import isin
import pandas as pd

week = 17
positions = ['DST', 'FLEX', 'QB', 'RB', 'TE', 'WR']
week_salaries_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{week}.csv")
week_salaries_df['Name'] = week_salaries_df['Name'].str.strip()

week_contest_standings_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-main-slate-contest-standings-week{week}.csv")
week_contest_standings_df['Player'] = week_contest_standings_df['Player'].str.strip()

class Player:
    def __init__(self, position, name):
        self.position = position
        self.name = name

class PlayerContestResult:
    def __init__(self, percent_drafted, fantasy_points):
        self.percent_drafted = percent_drafted
        self.fantasy_points = fantasy_points

def get_player_contest_result(player_name):
  filtered_df = week_contest_standings_df[week_contest_standings_df['Player'] == player_name]

  if len(filtered_df) > 0:
    return PlayerContestResult(filtered_df['%Drafted'].iloc[0], filtered_df['FPTS'].iloc[0])

  return PlayerContestResult('', '0%', '0')

def get_player_salary(player_name):
  filtered_df = week_salaries_df[week_salaries_df["Name"] == player_name]

  if len(filtered_df) > 0:
    return int(filtered_df['Salary'].iloc[0])

  return 0

def get_entry_lineup(lineup_position_and_players):
  prefix_delimitier = ''
  for position in positions:
    lineup_position_and_players = lineup_position_and_players.replace(position, f'{prefix_delimitier}{position},')
    prefix_delimitier = ','

  lineup_values = lineup_position_and_players.split(',')

  lineup = []
  if len(lineup_values) == 18:
    for i in range(1, 18, 2):
      lineup.append(Player(lineup_values[i-1].strip(), lineup_values[i].strip()))

  return lineup


# DST Bills FLEX Justin Jefferson QB Kirk Cousins RB De'Von Achane RB Jerome Ford TE Kyle Pitts WR Keenan Allen WR Amari Cooper WR Adam Thielen

num_top_rows = 10

for index, row in week_contest_standings_df.head(num_top_rows).iterrows():
  entry_lineup = get_entry_lineup(row['Lineup'])

  lineup_df = pd.DataFrame(columns=['Position', 'Name', 'Salary', '% Drafted', 'Exp Pts', 'Act Pts', 'Diff'])
  lineup_salary = 0
  lineup_ownership_percent = 0.0
  lineup_expected_points = 0.0
  lineup_actual_points = 0.0
  lineup_actual_expected_points_difference = 0.0

  for player in entry_lineup:
    salary = get_player_salary(player.name)
    expected_points = (salary * 4) / int(1000)
    player_contest_outcome = get_player_contest_result(player.name)
    actual_expected_points_difference = round(player_contest_outcome.fantasy_points - expected_points, 2)

    lineup_salary += salary
    lineup_ownership_percent += float(player_contest_outcome.percent_drafted.replace('%', ''))
    lineup_expected_points += expected_points
    lineup_actual_points += player_contest_outcome.fantasy_points
    lineup_actual_expected_points_difference += actual_expected_points_difference

    #row = [player.position, player.name, salary, player_contest_outcome.percent_drafted, expected_points, player_contest_outcome.fantasy_points, actual_expected_points_difference]

    #lineup_df = lineup_df.append(pd.Series(row, index=lineup_df.columns), ignore_index=True)
    #lineup_df = pd.concat(row)
    #lineup_df = pd.concat([lineup_df, pd.DataFrame(row)], axis=0, ignore_index=True)
    #lineup_df = pd.concat([lineup_df, pd.DataFrame.from_records(row)])
    #lineup_df = pd.concat([pd.Series(row, index=lineup_df.columns)])
    row = {'Position': [player.position], 'Name': [player.name], 'Salary': [salary], '% Drafted': [player_contest_outcome.percent_drafted], 'Exp Pts': [expected_points], 'Act Pts': [player_contest_outcome.fantasy_points], 'Diff': [actual_expected_points_difference]}
    lineup_df = pd.concat([lineup_df, pd.DataFrame(row)], axis=0, ignore_index=True)

  #total_row = ['', '', lineup_salary, f'{round(lineup_ownership_percent, 2)}%', lineup_expected_points, lineup_actual_points, lineup_actual_expected_points_difference]
  #lineup_df = lineup_df.append(pd.Series(total_row, index=lineup_df.columns), ignore_index=True)

  total_row = {'Position': '', 'Name': '', 'Salary': [lineup_salary], '% Drafted': [f'{round(lineup_ownership_percent, 2)}%'], 'Exp Pts': [lineup_expected_points], 'Act Pts': [lineup_actual_points], 'Diff': [lineup_actual_expected_points_difference]}
  lineup_df = pd.concat([lineup_df, pd.DataFrame(total_row)], axis=0, ignore_index=True)

  #lineup_df.style.applymap('text-align: left', subset=['Name'])
  lineup_df.to_string(justify='left', index=False)
  lineup_df['Name'] = lineup_df['Name'].apply(lambda x: x.ljust(20))
  print(lineup_df)
  print('')



  Position                  Name Salary % Drafted  Exp Pts  Act Pts   Diff
0      DST  Texans                 3000     6.67%     12.0    21.00   9.00
1     FLEX  DJ Moore               6600     4.24%     26.4    33.90   7.50
2       QB  Lamar Jackson          7700     6.74%     30.8    39.34   8.54
3       RB  De'Von Achane          6700     8.71%     26.8    26.70  -0.10
4       RB  Najee Harris           5000     4.20%     20.0    27.20   7.20
5       TE  Isaiah Likely          4600     7.72%     18.4    18.20  -0.20
6       WR  Brandon Aiyuk          7000    13.05%     28.0    27.40  -0.60
7       WR  Zay Flowers            5600    12.16%     22.4    22.60   0.20
8       WR  Cedrick Wilson Jr.     3600     8.66%     14.4    10.70  -3.70
9                                 49800    72.15%    199.2   227.04  27.84

  Position                  Name Salary % Drafted  Exp Pts  Act Pts  Diff
0      DST  Texans                 3000     6.67%     12.0     21.0   9.0
1     FLEX  Davante Adams 